In [1]:
pip install tensorflow numpy pillow


  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import kagglehub

path = kagglehub.dataset_download("bulentsiyah/semantic-drone-dataset")
print("Path to dataset files:", path)


c:\Users\micha\anaconda3\envs\semantic-segmentation\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\micha\.cache\kagglehub\datasets\bulentsiyah\semantic-drone-dataset\versions\6


In [ ]:

class SegmentationGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, mask_dir, batch_size, image_size):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_filenames = os.listdir(image_dir)
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, idx):
        batch_files = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        images, masks = [], []

        for filename in batch_files:
            img = Image.open(os.path.join(self.image_dir, filename)).resize(self.image_size)
            mask_name = filename.replace('.jpg', '_mask.png')
            mask = Image.open(os.path.join(self.mask_dir, mask_name)).resize(self.image_size)

            images.append(np.array(img) / 255.0)
            masks.append(np.array(mask))

        return np.array(images), np.array(masks)

# Instantiate generator
images_path = r"C:\Users\micha\.cache\kagglehub\datasets\bulentsiyah\semantic-drone-dataset\versions\6\dataset\semantic_drone_dataset\original_images"
mask_path   = r"C:\Users\micha\.cache\kagglehub\datasets\bulentsiyah\semantic-drone-dataset\versions\6\RGB_color_image_masks\RGB_color_image_masks"

train_gen = SegmentationGenerator(images_path, mask_path, batch_size=4, image_size=(256, 256))
